In [1]:
import pandas as pd
from datetime import datetime

**Reading the files and inspecting**

there are date time object in the faults will read the csv again.`

In [109]:
# faults = pd.read_csv("../data/J1939Faults.csv") 
# this call gave warning 
# donot ignore warning low_memory=False as you loose lot of rows.
# Also it has date time so read using datetime we may need it later.

faults = pd.read_csv("../data/J1939Faults.csv", low_memory=False,
                    parse_dates=['EventTimeStamp', 'LocationTimeStamp'])
faults.shape #(1187335, 20)
faults.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1187335 entries, 0 to 1187334
Data columns (total 20 columns):
 #   Column                 Non-Null Count    Dtype         
---  ------                 --------------    -----         
 0   RecordID               1187335 non-null  int64         
 1   ESS_Id                 1187335 non-null  int64         
 2   EventTimeStamp         1187335 non-null  datetime64[ns]
 3   eventDescription       1126490 non-null  object        
 4   actionDescription      0 non-null        float64       
 5   ecuSoftwareVersion     891285 non-null   object        
 6   ecuSerialNumber        844318 non-null   object        
 7   ecuModel               1122577 non-null  object        
 8   ecuMake                1122577 non-null  object        
 9   ecuSource              1187335 non-null  int64         
 10  spn                    1187335 non-null  int64         
 11  fmi                    1187335 non-null  int64         
 12  active                 11873

In [41]:
diagnostic = pd.read_csv("../data/VehicleDiagnosticOnboardData.csv")
diagnostic.shape #12821626 rows x 4 columns
diagnostic.info

<bound method DataFrame.info of                 Id                      Name        Value  FaultId
0                1                 IgnStatus        False        1
1                2         EngineOilPressure            0        1
2                3      EngineOilTemperature     96.74375        1
3                4        TurboBoostPressure            0        1
4                5                EngineLoad           11        1
...            ...                       ...          ...      ...
12821621  12864020  EngineCoolantTemperature        181.4  1248457
12821622  12864021              ParkingBrake        False  1248457
12821623  12864022    SwitchedBatteryVoltage         14.1  1248457
12821624  12864023               DistanceLtd  28606.65625  1248457
12821625  12864024                LampStatus         1023  1248458

[12821626 rows x 4 columns]>

In [31]:
service = pd.read_excel("../data/Service Fault Codes_1_0_0_167.xlsx")
service.shape #(7124, 14)

C:\Users\upadh\anaconda3\lib\site-packages\openpyxl\worksheet\_read_only.py:79: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():


(7124, 14)

**EDA for the faults table**

****SUMMARY****
1. Faults table have 1187335 (1.18X10^6) obseravtions with  20 feactures.
2. 20 features are
['RecordID',
 'ESS_Id',
 'EventTimeStamp',
 'eventDescription',
 'actionDescription',
 'ecuSoftwareVersion',
 'ecuSerialNumber',
 'ecuModel',
 'ecuMake',
 'ecuSource',
 'spn',
 'fmi',
 'active',
 'activeTransitionCount',
 'faultValue',
 'EquipmentID',
 'MCTNumber',
 'Latitude',
 'Longitude',
 'LocationTimeStamp']
 
 3. there are NaN in the columns
 RecordID                       0
ESS_Id                         0
EventTimeStamp                 0
eventDescription           60845
actionDescription        1187335
ecuSoftwareVersion        296050
ecuSerialNumber           343017
ecuModel                   64758
ecuMake                    64758
ecuSource                      0
spn                            0
fmi                            0
active                         0
activeTransitionCount          0
faultValue               1187335
EquipmentID                    0
MCTNumber                      0
Latitude                       0
Longitude                      0
LocationTimeStamp              0

4. As "actionDescription" and "faultValue" are all nan we will drop these columns.
5. we will use the lat long column tio find the trucks that are in 1mile radius of one of the three service stations, we will drop 0.01 degree from both lat and long.
6. One degree of latitude equals approximately 364,000 feet (69 miles). One-degree of longitude equals 288,200 feet (54.6 miles).
7. we will also remove the "EquipmentID " which are more than 5 digits.
    there are 2169 rows and leaves 1185166 observation.
    after removing we have 1045 trucks in the dataset.
8. There are some timestamps that are also beyond the time window mentioned we will remove them, as mentioned by Joshua.
9. Also the active code which are False means that the fault code is fixed now.
 

In [3]:
faults.head(5)

,RecordID,ESS_Id,EventTimeStamp,eventDescription,actionDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,fmi,active,activeTransitionCount,faultValue,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp
0,1,990349,2015-02-21 10:47:13,Low (Severity Low) Engine Coolant Level,NaN,unknown,unknown,unknown,unknown,0,111,17,True,2,NaN,1439,105354361,38.857638,-84.626851,2015-02-21 11:34:25
1,2,990360,2015-02-21 11:34:34,NaN,NaN,unknown,unknown,unknown,unknown,11,629,12,True,127,NaN,1439,105354361,38.857638,-84.626851,2015-02-21 11:35:10
2,3,990364,2015-02-21 11:35:31,Incorrect Data Steering Wheel Angle,NaN,unknown,unknown,unknown,unknown,11,1807,2,False,127,NaN,1369,105336226,41.421250,-87.767361,2015-02-21 11:35:26
3,4,990370,2015-02-21 11:35:33,Incorrect Data Steering Wheel Angle,NaN,unknown,unknown,unknown,unknown,11,1807,2,True,127,NaN,1369,105336226,41.421018,-87.767361,2015-02-21 11:36:08
4,5,990416,2015-02-21 11:39:41,NaN,NaN,22281684P01*22357957P01*22362082P01*,13063430,0USA13_13_0415_2238A,VOLVO,0,4364,17,False,2,NaN,1674,105427130,38.416481,-89.442638,2015-02-21 11:39:37


In [45]:
faults.isna().sum()

RecordID                       0
ESS_Id                         0
EventTimeStamp                 0
eventDescription           60845
actionDescription        1187335
ecuSoftwareVersion        296050
ecuSerialNumber           343017
ecuModel                   64758
ecuMake                    64758
ecuSource                      0
spn                            0
fmi                            0
active                         0
activeTransitionCount          0
faultValue               1187335
EquipmentID                    0
MCTNumber                      0
Latitude                       0
Longitude                      0
LocationTimeStamp              0
dtype: int64

In [110]:
# as the actionDescription and faultValue  are all NaN values will drop them.

faults = faults.drop(columns = ['actionDescription', 'faultValue'])
faults

,RecordID,ESS_Id,EventTimeStamp,eventDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,fmi,active,activeTransitionCount,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp
0,1,990349,2015-02-21 10:47:13,Low (Severity Low) Engine Coolant Level,unknown,unknown,unknown,unknown,0,111,17,True,2,1439,105354361,38.857638,-84.626851,2015-02-21 11:34:25
1,2,990360,2015-02-21 11:34:34,NaN,unknown,unknown,unknown,unknown,11,629,12,True,127,1439,105354361,38.857638,-84.626851,2015-02-21 11:35:10
2,3,990364,2015-02-21 11:35:31,Incorrect Data Steering Wheel Angle,unknown,unknown,unknown,unknown,11,1807,2,False,127,1369,105336226,41.421250,-87.767361,2015-02-21 11:35:26
3,4,990370,2015-02-21 11:35:33,Incorrect Data Steering Wheel Angle,unknown,unknown,unknown,unknown,11,1807,2,True,127,1369,105336226,41.421018,-87.767361,2015-02-21 11:36:08
4,5,990416,2015-02-21 11:39:41,NaN,22281684P01*22357957P01*22362082P01*,13063430,0USA13_13_0415_2238A,VOLVO,0,4364,17,False,2,1674,105427130,38.416481,-89.442638,2015-02-21 11:39:37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1187330,1248454,123904424,2020-03-06 14:00:26,Low (Severity Low) Catalyst Tank Level,04384413*22383729*082218154102*60701732*G1*BGT*,80156139,6X1u17D1500000000,CMMNS,0,1761,17,False,3,2282,105439740,37.094768,-85.897407,2020-03-06 14:00:21
1187331,1248455,123905139,2020-03-06 14:04:23,Condition Exists Engine Protection Torque Derate,04358814*06099720*030816202706*09400153*G1*BDR*,79932020,6X1u13D1500000000,CMMNS,0,1569,31,True,5,1994,105354084,34.390740,-79.461805,2020-03-06 14:04:59
1187332,1248456,123905996,2020-03-06 14:13:38,Abnormal Rate of Change Aftertreatment 1 Intak...,05317106*05100987*050719120655*09401585*G1*BDR*,79880653,6X1u13D1500000000,CMMNS,0,3216,10,True,1,1850,105336308,34.430370,-84.920509,2020-03-06 14:14:14
1187333,1248457,123906113,2020-03-06 14:14:13,Low (Severity Medium) Engine Coolant Level,04384413*22544852*090619141107*60701756*G1*BGT*,NaN,NaN,NaN,0,111,18,True,8,2377,108605700,35.030925,-85.321527,2020-03-06 14:14:49


In [128]:
#total number of rows more than 5
len(faults[faults['EquipmentID'].str.len() < 5])

1185143

In [112]:
#cleaning the faults data to hav equipmentID less than or equall to 5
#faults = 1187335
# faults = faults[faults['EquipmentID'].str.len() <= 5]
faults = faults[faults['EquipmentID'].str.len() < 5]
faults.shape #1185143, 18

#there are 1042 trucks which have ID less than 5 and 
# 3 that are equal to 5 with alpha numeric chaarcters.

(1185143, 18)

Note we need to find the rows which are 0.01 mile from the service stations and drop them.

We do not have the target variable column in the data set so we need to find four groups which never derate, which have 1526 but no 5246,
some with 1526 and 5246 and some with 5246 only.

In [113]:
#find the rows close to service stations within 0.01 mile then droping them

faults['latA_diff'] = abs(faults['Latitude']-36.0666667)
faults['latB_diff'] = abs(faults['Latitude']-35.5883333)
faults['latC_diff'] = abs(faults['Latitude']-36.1950)
faults['longA_diff'] = abs(faults['Longitude']-(-86.4347222))
faults['longB_diff'] = abs(faults['Longitude']-(-86.4438888))
faults['longC_diff'] = abs(faults['Longitude']-(-83.174722))

faults

#abs() gives the result as absolute number
# now drop the rows which have values less than 0.01, meaning in proximity of the service station.

C:\Users\upadh\AppData\Local\Temp\ipykernel_12880\2039169360.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  faults['latA_diff'] = abs(faults['Latitude']-36.0666667)
C:\Users\upadh\AppData\Local\Temp\ipykernel_12880\2039169360.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  faults['latB_diff'] = abs(faults['Latitude']-35.5883333)
C:\Users\upadh\AppData\Local\Temp\ipykernel_12880\2039169360.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .lo

,RecordID,ESS_Id,EventTimeStamp,eventDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,...,MCTNumber,Latitude,Longitude,LocationTimeStamp,latA_diff,latB_diff,latC_diff,longA_diff,longB_diff,longC_diff
0,1,990349,2015-02-21 10:47:13,Low (Severity Low) Engine Coolant Level,unknown,unknown,unknown,unknown,0,111,...,105354361,38.857638,-84.626851,2015-02-21 11:34:25,2.790971,3.269305,2.662638,1.807871,1.817038,1.452129
1,2,990360,2015-02-21 11:34:34,NaN,unknown,unknown,unknown,unknown,11,629,...,105354361,38.857638,-84.626851,2015-02-21 11:35:10,2.790971,3.269305,2.662638,1.807871,1.817038,1.452129
2,3,990364,2015-02-21 11:35:31,Incorrect Data Steering Wheel Angle,unknown,unknown,unknown,unknown,11,1807,...,105336226,41.421250,-87.767361,2015-02-21 11:35:26,5.354583,5.832917,5.226250,1.332639,1.323472,4.592639
3,4,990370,2015-02-21 11:35:33,Incorrect Data Steering Wheel Angle,unknown,unknown,unknown,unknown,11,1807,...,105336226,41.421018,-87.767361,2015-02-21 11:36:08,5.354351,5.832685,5.226018,1.332639,1.323472,4.592639
4,5,990416,2015-02-21 11:39:41,NaN,22281684P01*22357957P01*22362082P01*,13063430,0USA13_13_0415_2238A,VOLVO,0,4364,...,105427130,38.416481,-89.442638,2015-02-21 11:39:37,2.349814,2.828148,2.221481,3.007916,2.998749,6.267916
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1187330,1248454,123904424,2020-03-06 14:00:26,Low (Severity Low) Catalyst Tank Level,04384413*22383729*082218154102*60701732*G1*BGT*,80156139,6X1u17D1500000000,CMMNS,0,1761,...,105439740,37.094768,-85.897407,2020-03-06 14:00:21,1.028101,1.506435,0.899768,0.537315,0.546482,2.722685
1187331,1248455,123905139,2020-03-06 14:04:23,Condition Exists Engine Protection Torque Derate,04358814*06099720*030816202706*09400153*G1*BDR*,79932020,6X1u13D1500000000,CMMNS,0,1569,...,105354084,34.390740,-79.461805,2020-03-06 14:04:59,1.675927,1.197593,1.804260,6.972917,6.982084,3.712917
1187332,1248456,123905996,2020-03-06 14:13:38,Abnormal Rate of Change Aftertreatment 1 Intak...,05317106*05100987*050719120655*09401585*G1*BDR*,79880653,6X1u13D1500000000,CMMNS,0,3216,...,105336308,34.430370,-84.920509,2020-03-06 14:14:14,1.636297,1.157963,1.764630,1.514213,1.523380,1.745787
1187333,1248457,123906113,2020-03-06 14:14:13,Low (Severity Medium) Engine Coolant Level,04384413*22544852*090619141107*60701756*G1*BGT*,NaN,NaN,NaN,0,111,...,108605700,35.030925,-85.321527,2020-03-06 14:14:49,1.035742,0.557408,1.164075,1.113195,1.122362,2.146805


In [71]:
faults['EquipmentID'].value_counts().to_frame() #1042

,EquipmentID
1641,17492
1605,16393
1646,15462
1618,14986
1606,14973
...,...
2274,2
2298,2
2356,2
2381,2


In [73]:
faults['EquipmentID'].nunique() #1045

1042

In [9]:
faults[(faults['spn'] == 1569) & (faults['fmi'] == 31)] #10717 with and without fmi=31

faults[(faults['spn'] == 1569) & (faults['fmi'] == 31)]['EquipmentID'].nunique() #498

498

In [10]:
faults.loc[~(faults['spn'].isin([1569, 5426]))] #1174449
# faults.loc[~(faults['spn'] .isin([1569, 5426]))]['EquipmentID'].nunique() #1045

,RecordID,ESS_Id,EventTimeStamp,eventDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,fmi,active,activeTransitionCount,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp
0,1,990349,2015-02-21 10:47:13,Low (Severity Low) Engine Coolant Level,unknown,unknown,unknown,unknown,0,111,17,True,2,1439,105354361,38.857638,-84.626851,2015-02-21 11:34:25
1,2,990360,2015-02-21 11:34:34,NaN,unknown,unknown,unknown,unknown,11,629,12,True,127,1439,105354361,38.857638,-84.626851,2015-02-21 11:35:10
2,3,990364,2015-02-21 11:35:31,Incorrect Data Steering Wheel Angle,unknown,unknown,unknown,unknown,11,1807,2,False,127,1369,105336226,41.421250,-87.767361,2015-02-21 11:35:26
3,4,990370,2015-02-21 11:35:33,Incorrect Data Steering Wheel Angle,unknown,unknown,unknown,unknown,11,1807,2,True,127,1369,105336226,41.421018,-87.767361,2015-02-21 11:36:08
4,5,990416,2015-02-21 11:39:41,NaN,22281684P01*22357957P01*22362082P01*,13063430,0USA13_13_0415_2238A,VOLVO,0,4364,17,False,2,1674,105427130,38.416481,-89.442638,2015-02-21 11:39:37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1187329,1248453,123901925,2020-03-06 13:44:45,Low (Severity Medium) Engine Coolant Level,04358814*06030918*051718174436*09401683*G1*BDR*,79904453,6X1u13D1500000000,CMMNS,0,111,18,False,93,1886,105351219,39.013472,-77.037685,2020-03-06 13:44:41
1187330,1248454,123904424,2020-03-06 14:00:26,Low (Severity Low) Catalyst Tank Level,04384413*22383729*082218154102*60701732*G1*BGT*,80156139,6X1u17D1500000000,CMMNS,0,1761,17,False,3,2282,105439740,37.094768,-85.897407,2020-03-06 14:00:21
1187332,1248456,123905996,2020-03-06 14:13:38,Abnormal Rate of Change Aftertreatment 1 Intak...,05317106*05100987*050719120655*09401585*G1*BDR*,79880653,6X1u13D1500000000,CMMNS,0,3216,10,True,1,1850,105336308,34.430370,-84.920509,2020-03-06 14:14:14
1187333,1248457,123906113,2020-03-06 14:14:13,Low (Severity Medium) Engine Coolant Level,04384413*22544852*090619141107*60701756*G1*BGT*,NaN,NaN,NaN,0,111,18,True,8,2377,108605700,35.030925,-85.321527,2020-03-06 14:14:49


In [11]:
faults[(faults['spn']==5246)] #1189
# faults[(faults['spn']==5246)]['EquipmentID'].nunique() #211

# same number of rows as for 1569 which is 10717 rows × 18 columns

,RecordID,ESS_Id,EventTimeStamp,eventDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,fmi,active,activeTransitionCount,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp
45,46,990931,2015-02-21 12:10:51,NaN,04993120*00027849*082113134117*07700053*I0*BBZ*,79464664,6X1u10D1500000000,CMMNS,0,5246,0,True,1,1395,105349612,36.065972,-86.433425,2015-02-21 12:11:27
1918,1919,1007751,2015-02-22 19:44:55,NaN,04993120*00027849*082113134117*07700053*I0*BBZ*,79464664,6X1u10D1500000000,CMMNS,0,5246,0,True,1,1395,105349612,36.066203,-86.434814,2015-02-22 19:46:27
2058,2059,1010486,2015-02-23 04:00:21,NaN,04993120*00027849*082113134117*07700053*I0*BBZ*,79464664,6X1u10D1500000000,CMMNS,0,5246,0,False,1,1395,105349612,36.066666,-86.434537,2015-02-23 01:06:06
2089,2090,1011009,2015-02-23 05:05:44,NaN,05290170*03015749*051914190353*09400015*G1*BDR*,79642446,6X1u13D1500000000,CMMNS,0,5246,0,True,1,1630,105329900,40.733009,-74.087777,2015-02-23 05:08:23
2971,2972,1026305,2015-02-23 15:54:22,NaN,unknown,unknown,unknown,unknown,0,5246,0,True,1,1487,105369355,28.077361,-81.897083,2015-02-23 15:54:58
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1183032,1244156,121610128,2020-02-19 07:02:33,NaN,05317106*05005224*051718172255*09401583*G1*BDR*,79845785,6X1u13D1500000000,CMMNS,0,5246,0,True,1,1814,105369518,36.067037,-86.434120,2020-02-19 07:03:09
1183684,1244808,121909497,2020-02-21 07:23:44,NaN,04384413*22246857*090619141107*60701756*G1*BGT*,80092582,6X1u17D1500000000,CMMNS,0,5246,16,True,1,2211,105329862,36.066296,-86.434305,2020-02-21 07:24:20
1184328,1245452,122305094,2020-02-24 15:28:05,NaN,04384413*22246857*090619141107*60701756*G1*BGT*,80092582,6X1u17D1500000000,CMMNS,0,5246,16,False,1,2211,105329862,36.066620,-86.434722,2020-02-24 15:28:01
1184330,1245454,122305096,2020-02-24 15:27:26,NaN,04384413*22246857*090619141107*60701756*G1*BGT*,80092582,6X1u17D1500000000,CMMNS,0,5246,0,True,1,2211,105329862,36.066620,-86.434722,2020-02-24 15:28:02


In [12]:
(10717+1174449 +1189 )

1186355

In [114]:
#droping the record that happens to be in proximity to any three service stations (0.01 miles)
faults_prep = (faults.loc[((faults['latA_diff']) > 0.01) &
            ((faults['latB_diff']) > 0.01) &
            ((faults['latC_diff']) > 0.01) &
            ((faults['longA_diff']) > 0.01) &
            ((faults['longB_diff']) > 0.01) &
            ((faults['longC_diff']) > 0.01)]
).drop(['latA_diff', 'latB_diff', 'latC_diff', 'longA_diff', 'longB_diff', 'longC_diff'], axis=1)



In [117]:
#drop the illegitimate row with wrong  EventTimeStamp
# faults_prep.loc[faults['EventTimeStamp'].dt.year > 2011].shape
faults_prep = faults_prep.loc[(faults_prep['EventTimeStamp']) > "2014-12-31"] #going to use date and time after 2011.

In [118]:
# faults.shap/e #(1185166, 24)
faults_prep.shape #

(1030183, 18)

In [119]:
faults_prep['active'].value_counts()

True     533192
False    496991
Name: active, dtype: int64

Now as we have two active codes True and False we are removing the False which means that the fault is fixed

In [120]:
faults_clean = faults_prep.loc[(faults_prep['active']) != False] #533192 rows × 18 columns

In [124]:
faults_clean.shape

(533192, 18)

In [105]:
# faults.loc[faults['spn'] == 5246]['fmi'].value_counts()
# faults.loc[faults['spn'] == 929]['fmi'].value_counts() #256516
faults_clean.loc[faults['spn'] == 929]

,RecordID,ESS_Id,EventTimeStamp,eventDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,fmi,active,activeTransitionCount,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp
28,29,990653,2015-02-21 11:53:52,Abnormal Update Rate Tire Location,unknown,unknown,unknown,unknown,49,929,9,True,126,1605,105415080,37.521898,-85.881759,2015-02-21 11:54:29
30,31,990689,2015-02-21 11:56:15,Abnormal Update Rate Tire Location,unknown,unknown,unknown,unknown,49,929,9,True,126,1634,105442858,37.521064,-85.881851,2015-02-21 11:56:52
81,82,991323,2015-02-21 12:33:48,Abnormal Update Rate Tire Location,unknown,unknown,unknown,unknown,49,929,9,True,126,1646,105420184,36.137546,-85.265185,2015-02-21 12:34:25
95,96,991581,2015-02-21 12:49:35,Abnormal Update Rate Tire Location,unknown,unknown,unknown,unknown,49,929,9,True,126,1610,105415630,35.457592,-86.050231,2015-02-21 12:50:10
105,106,991708,2015-02-21 12:51:45,Abnormal Update Rate Tire Location,unknown,unknown,unknown,unknown,49,929,9,True,126,1620,105416411,36.097638,-86.420138,2015-02-21 12:58:31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
968065,998573,45844626,2018-04-16 01:38:02,Abnormal Update Rate Tire Location,NaN,NaN,NaN,NaN,49,929,9,True,126,1649,105417985,35.978379,-85.015648,2018-04-16 01:38:39
968090,998598,45847272,2018-04-16 04:00:03,Abnormal Update Rate Tire Location,NaN,NaN,NaN,NaN,49,929,9,True,126,1649,105417985,36.174398,-86.352268,2018-04-16 03:46:13
969712,1001131,46259777,2018-04-19 10:08:40,Abnormal Update Rate Tire Location,NaN,NaN,NaN,NaN,49,929,9,True,126,1649,105417985,35.869768,-86.460185,2018-04-19 10:26:18
969869,1001288,46291775,2018-04-19 13:28:48,Abnormal Update Rate Tire Location,NaN,NaN,CECU3B-NAMUX4,PACCR,49,929,9,True,126,1644,105435663,35.794120,-86.399490,2018-04-19 14:05:58


In [104]:
faults_clean.loc[faults['spn'] == 1569]['fmi'].value_counts()

31    4922
Name: fmi, dtype: int64

In [106]:
faults_clean.loc[faults['fmi'] == 16]['spn'].value_counts()
# faults.loc[faults['fmi'] == 16]['spn'].nunique() #51

3251     558
1231     480
3058     465
81       418
102      264
5939     235
97       180
101      179
3226     143
411      134
1209     134
175       93
2659      84
3363      80
5246      76
1239      38
5442      36
103       27
95        25
157       24
6773      20
4334      20
168       15
4765      14
444       12
3242      11
2795      11
110       10
1347       8
5862       8
4363       8
412        7
653        7
3216       7
5742       6
3936       6
5625       3
3246       3
5615       2
4331       2
188        2
3064       2
190        2
4360       2
1127       1
6146       1
5614       1
3217       1
88121      1
Name: spn, dtype: int64

In [121]:
faults_clean['EquipmentID'].value_counts().sort_values(ascending=False).to_frame() #there are a total of 1038 equipment

,EquipmentID
1692,9967
1641,7557
1605,7157
1606,6828
1646,6785
...,...
2298,1
2274,1
2238,1
2356,1


In [122]:
faults_clean[faults['spn'] == 5246] #faults had 1189 rows for spn 5246 while upon cleaning we now have 485 rows.

C:\Users\upadh\AppData\Local\Temp\ipykernel_12880\1742060033.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  faults_clean[faults['spn'] == 5246] #1189 rows


,RecordID,ESS_Id,EventTimeStamp,eventDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,fmi,active,activeTransitionCount,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp
2089,2090,1011009,2015-02-23 05:05:44,NaN,05290170*03015749*051914190353*09400015*G1*BDR*,79642446,6X1u13D1500000000,CMMNS,0,5246,0,True,1,1630,105329900,40.733009,-74.087777,2015-02-23 05:08:23
2971,2972,1026305,2015-02-23 15:54:22,NaN,unknown,unknown,unknown,unknown,0,5246,0,True,1,1487,105369355,28.077361,-81.897083,2015-02-23 15:54:58
5713,5714,1070646,2015-02-25 13:53:08,NaN,unknown,unknown,unknown,unknown,0,5246,0,True,1,1329,105400037,39.399583,-82.974768,2015-02-25 13:56:31
6534,6535,1097942,2015-02-26 22:24:29,NaN,04993120*00021657*082113134117*07700053*I0*BBZ*,79466573,6X1u10D1500000000,CMMNS,0,5246,0,True,1,1419,105355995,37.596805,-85.865555,2015-02-26 22:25:05
6628,6629,1104361,2015-02-27 09:09:56,NaN,04993120*00054166*082113134117*07700053*I0*BBZ*,79487658,6X1u10D1500000000,CMMNS,0,5246,0,True,1,1486,105338831,40.534259,-76.431805,2015-02-27 09:10:33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1173036,1232200,116794344,2020-01-13 13:18:31,NaN,NaN,NaN,NaN,NaN,49,5246,19,True,46,302,105418777,38.192824,-85.859490,2020-01-13 13:19:06
1178571,1238712,119571469,2020-02-03 15:46:46,NaN,04384413*22246857*121817205924*60701721*G1*BGT*,80092582,6X1u17D1500000000,CMMNS,0,5246,16,True,1,2211,105329862,35.833935,-86.410925,2020-02-03 15:47:23
1179414,1239555,119959276,2020-02-06 07:45:08,NaN,04358814*06005963*051718174436*09401683*G1*BDR*,79897320,6X1u13D1500000000,CMMNS,0,5246,0,True,1,1854,105385876,35.943611,-83.823009,2020-02-06 07:45:44
1181700,1241841,120905759,2020-02-13 13:32:39,NaN,04358814*06026985*051718174436*09401683*G1*BDR*,79903054,6X1u13D1500000000,CMMNS,0,5246,0,True,1,1872,105301976,35.707268,-81.397037,2020-02-13 13:33:15


In [130]:
faults_clean[(faults_clean['EquipmentID'] == '1499') & (faults['spn'] == 5246)]

C:\Users\upadh\AppData\Local\Temp\ipykernel_12880\2971066359.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  faults_clean[(faults_clean['EquipmentID'] == '1499') & (faults['spn'] == 5246)]


,RecordID,ESS_Id,EventTimeStamp,eventDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,fmi,active,activeTransitionCount,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp


**EDA for diagnostics**

In [22]:
diagnostic.head(5)

,Id,Name,Value,FaultId
0,1,IgnStatus,False,1
1,2,EngineOilPressure,0,1
2,3,EngineOilTemperature,96.74375,1
3,4,TurboBoostPressure,0,1
4,5,EngineLoad,11,1


In [23]:
diagnostic.isna().sum()

Id         0
Name       0
Value      0
FaultId    0
dtype: int64

In [24]:
diagnostic.info

<bound method DataFrame.info of                 Id                      Name        Value  FaultId
0                1                 IgnStatus        False        1
1                2         EngineOilPressure            0        1
2                3      EngineOilTemperature     96.74375        1
3                4        TurboBoostPressure            0        1
4                5                EngineLoad           11        1
...            ...                       ...          ...      ...
12821621  12864020  EngineCoolantTemperature        181.4  1248457
12821622  12864021              ParkingBrake        False  1248457
12821623  12864022    SwitchedBatteryVoltage         14.1  1248457
12821624  12864023               DistanceLtd  28606.65625  1248457
12821625  12864024                LampStatus         1023  1248458

[12821626 rows x 4 columns]>

In [25]:
diagnostic.dtypes

Id          int64
Name       object
Value      object
FaultId     int64
dtype: object

In [26]:
diagnostic['Value'].value_counts().to_frame().reset_index().head(50)
# 12821626 - 1009465

,index,Value
0,0,981492
1,False,793736
2,True,789606
3,100,417610
4,1023,353059
5,1279,314513
6,255,301633
7,32,291368
8,66.48672,278448
9,64.6226,130898


In [27]:
diagnostic['FaultId'].value_counts().to_frame().reset_index()

diagnostic['FaultId'].nunique()

1187335

In [28]:
diagnostic['Name'].value_counts().to_frame().reset_index()
# diagnostic['Name'].nunique()

,index,Name
0,LampStatus,1187335
1,IgnStatus,608454
2,EngineRpm,586921
3,IntakeManifoldTemperature,586291
4,EngineOilPressure,586244
5,EngineCoolantTemperature,586071
6,BarometricPressure,585976
7,DistanceLtd,585819
8,EngineLoad,585621
9,FuelRate,585237


In [30]:
diagnostic.head(5)

,Id,Name,Value,FaultId
0,1,IgnStatus,False,1
1,2,EngineOilPressure,0,1
2,3,EngineOilTemperature,96.74375,1
3,4,TurboBoostPressure,0,1
4,5,EngineLoad,11,1


In [33]:
# diagnostic.pivot(columns='FaultId', index='Id', values=['Name', 'Value'])

diagnostic_pivot = diagnostic.pivot(columns='Name', index='FaultId', values= 'Value')

In [34]:
diagnostic_pivot.describe()

Name,AcceleratorPedal,BarometricPressure,CruiseControlActive,CruiseControlSetSpeed,DistanceLtd,EngineCoolantTemperature,EngineLoad,EngineOilPressure,EngineOilTemperature,EngineRpm,...,FuelTemperature,IgnStatus,IntakeManifoldTemperature,LampStatus,ParkingBrake,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure
count,531889,585976,574916,576458,585819,586071,585621,586244,583912,586921,...,299110,608454,586291,1187335,399972,215,583916,114059,420503,583351
unique,261,89,2,74,450984,155,103,248,4122,11471,...,128,2,152,37,2,196,13533,129,260,162
top,0,14.355,False,66.48672,0,185,0,0,215.9937,0,...,32,True,120.2,1023,False,-10774.576171875,0,3276.75,100,0.29
freq,277162,85896,524183,278248,592,69584,91147,43317,4142,45767,...,282642,605818,16739,353020,266917,8,199673,102982,286284,94904


In [35]:
diagnostic_pivot.info

<bound method DataFrame.info of Name    AcceleratorPedal BarometricPressure CruiseControlActive  \
FaultId                                                           
1                      0              14.21               False   
2                    NaN                NaN                 NaN   
3                    NaN                NaN                 NaN   
4                    NaN                NaN                 NaN   
5                    NaN                NaN                 NaN   
...                  ...                ...                 ...   
1248454              NaN                NaN                 NaN   
1248455              100               14.5                True   
1248456                0             14.355                True   
1248457              1.6            14.4275               False   
1248458              NaN                NaN                 NaN   

Name    CruiseControlSetSpeed  DistanceLtd EngineCoolantTemperature  \
FaultId                  

In [32]:
service.head(5)

,Published in CES 14602,Cummins Fault Code,Revision,PID,SID,MID,J1587 FMI,SPN,J1939 FMI,J2012 Pcode,Lamp Color,Lamp Device,Cummins Description,Algorithm Description
0,Y,111,167,Not Mapped,254,0,12,629,12,P0606,Red,Stop / Shutdown,Engine Control Module Critical Internal Failur...,Error internal to the ECM related to memory ha...
1,Y,112,167,Not Mapped,20,128,7,635,7,Not Mapped,Red,Stop / Shutdown,Engine Timing Actuator Driver Circuit - Mechan...,Mechanical failure in the engine timing actuat...
2,Y,113,167,Not Mapped,20,128,3,635,3,Not Mapped,Amber,Warning,Engine Timing Actuator Driver Circuit - Voltag...,High signal voltage detected at the engine tim...
3,Y,114,167,Not Mapped,20,128,4,635,4,Not Mapped,Amber,Warning,Engine Timing Actuator Driver Circuit - Voltag...,Low voltage detected at the engine timing actu...
4,Y,115,167,190,Not Mapped,Not Mapped,2,612,2,P0008,Red,Stop / Shutdown,Engine Magnetic Speed/Position Lost Both of Tw...,The ECM has detected that the primary and back...


# to get the rolling dates

data["Date"] = pd.to_datetime(data["Date"])
data["rolling_sum_2days"] = data.rolling(window = '2d', on = "Date").Sales.sum()